In [3]:
import numpy as np
import pandas as pd
import json
from pyyoutube import Api
import requests
from datetime import datetime
from dateutil import parser

In [2]:
data = json.load(open('research/output/youtube_data.json'))
credentials = json.load(open("research/input/creds.json"))
AUTH_CREDS = 'research/input/client_secret.json'
API_KEY = credentials["api_key"]
ANALYTICS_API_KEY = credentials["analytics_api_key"]
api = Api(api_key=API_KEY)
SCOPES = ['https://www.googleapis.com/auth/youtube.readonly']

In [3]:
channels = {}
for video, content in data.items():
    channel = content["channelTitle"]
    channel_id = content["channelId"]
    if channel not in channels:
        channels[channel] = {"id":channel_id, "videos":[]}
    channels[channel]["videos"].append({'video_id':video,
                                        'annots':content['annots'], 
                                        'publishedAt':content['publishedAt'],
                                        'viewCount': content['viewCount'],
                                        'likeCount': content['likeCount'],
                                        'commentCount': content['commentCount'],
                                        'sector': content['sector']
                                         })

In [4]:
for channel, content in channels.items():
    dt_list = []
    content_list = []
    for i in range(len(content["videos"])):
        t = channels[channel]["videos"][i]["publishedAt"]
        dt_object = parser.parse(t)
        dt_list.append(dt_object)
        content_list.append(i)
    y = sorted(dt_list)
    dt_list, content_list = zip(*sorted(zip(dt_list, content_list)))
    final = []
    for index in content_list:
        final.append(content["videos"][index])
    video = api.get_channel_info(channel_id=channels[channel]["id"]).items[0].to_dict()
    channels[channel]["title"] = video["snippet"]["title"]
    channels[channel]["region"] = video["brandingSettings"]["channel"]["country"]
    channels[channel]["channelViewCount"] = video["statistics"]["viewCount"]
    channels[channel]["subscriberCount"] = video["statistics"]["subscriberCount"]
    channels[channel]["totalUploads"] = video["statistics"]["videoCount"]
    channels[channel]["launchDate"] = video["snippet"]["publishedAt"]
    channels[channel]["videos"] = final

In [5]:
channels

{'ebay': {'id': 'UCcA6bjXsoqQZst5jlM39A7g',
  'videos': [{'video_id': 'vM4x26EinMQ',
    'annots': ['Overcoming Reactance', 'Foot in the Door'],
    'publishedAt': '2015-11-09T15:02:55Z',
    'viewCount': 287408,
    'likeCount': 25,
    'commentCount': 0,
    'sector': 'Technology'},
   {'video_id': 'MjeLXaZK2VI',
    'annots': ['Concreteness', 'Social Identity'],
    'publishedAt': '2016-01-12T23:18:25Z',
    'viewCount': 3174,
    'likeCount': 4,
    'commentCount': 0,
    'sector': 'Technology'},
   {'video_id': 'PGhG-LzJRHo',
    'annots': ['Social Identity'],
    'publishedAt': '2016-07-18T23:49:24Z',
    'viewCount': 234811,
    'likeCount': 13,
    'commentCount': 0,
    'sector': 'Technology'},
   {'video_id': 'IEpNYKP5vbU',
    'annots': ['Social Identity'],
    'publishedAt': '2016-07-19T00:03:31Z',
    'viewCount': 230815,
    'likeCount': 12,
    'commentCount': 0,
    'sector': 'Technology'},
   {'video_id': '9Q1Wvh-lalc',
    'annots': ['Authority', 'Concreteness', 'Soci

In [7]:
import os
import google.oauth2.credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

In [8]:
youtube_analytics = build('youtube', 'v3', developerKey=ANALYTICS_API_KEY)